In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        'kfp<2' \
                        'google-cloud-pipeline-components<2'

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

In [ ]:
! gcloud config list

In [ ]:
PROJECT_ID = "vertex-nirvana-poc"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

In [ ]:
BUCKET_URI = "gs://vertex-nirvana-poc"  # @param {type:"string"}

In [ ]:
BUCKET = f"gs://vertex-nirvana-poc/"

In [ ]:
SERVICE_ACCOUNT = "665554893092-compute@developer.gserviceaccount.com"

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
from typing import NamedTuple

import google.cloud.aiplatform as aip
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/intro".format(BUCKET_URI)

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [ ]:
from google.cloud import aiplatform as aip
from kfp.v2.dsl import (
    Artifact,
    Dataset,
    Input,
    Model,
    Output,
    ClassificationMetrics,
    component,
    pipeline,
)
from kfp.v2 import compiler

In [ ]:
from google_cloud_pipeline_components.v1.endpoint import EndpointCreateOp, ModelDeployOp
from google_cloud_pipeline_components.v1.model import ModelUploadOp


In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/intro".format(BUCKET_URI)

In [ ]:
from kfp import dsl

@dsl.component
def conditional_branch_op(output: str) -> str:
    """Conditionally branch based on the output value."""
    # Define your condition logic here
    if output == "Hello, world!":
        return "condition_met"
    else:
        return "condition_not_met"


In [ ]:
from typing import NamedTuple
import kfp
from kfp import dsl, components

# Define your component functions here

@components.func_to_container_op
# def hello_world(text: str) -> str:
#     print(text)
#     return text
#@func_to_container_op
def hello_world(text: str) -> str:
    return f"Hello, {text}!"


#     return step1.output , step2.output, step3.output, step4.output
@dsl.pipeline(name='Conditional Hello World Pipeline')
def conditional_hello_world_pipeline(input_condition: str) -> NamedTuple(
    'Outputs',
    [
        ('step1_output', str),
        ('step2_output', str),
        ('step3_output', str),
        ('step4_output', str),
    ]
):
    # Step 1: Call the hello_world function with "world" input
    step1 = hello_world("world")

    # Step 2: Use the input_condition parameter to control the branching
    with dsl.Condition(input_condition == "condition_met"):
        # If the input_condition is "condition_met", execute this branch
        step2 = hello_world("Alice")

    with dsl.Condition(input_condition == "condition_met"):
        # If the input_condition is "condition_met", execute this branch
        step3 = hello_world("Bob")

    # Step 4: Always execute this step
    step4 = hello_world("OpenAI")

    return step1.output, step2.output, step3.output, step4.output


@dsl.pipeline(
    name="intro-pipeline-unique",
    description="A simple intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(text: str = "hello world"):
    hw_task = hello_world(text)
    # two_outputs_task = two_outputs(text)
    conditional_hello_world_pipeline_task = conditional_hello_world_pipeline(hw_task.output)
    # consumer_task = consumer(
    #     hw_task.output,
    #     two_outputs_task.outputs["output_one"],
    #     two_outputs_task.outputs["output_two"],
    # )

if __name__ == '__main__':
    compiler.Compiler().compile(pipeline_func=pipeline, package_path="Conditional_pipeline_job_with_caching.json")
    DISPLAY_NAME = "Conditional_pipeline_job_with_caching"
    job = aip.PipelineJob(
        display_name=DISPLAY_NAME,
        template_path="Conditional_pipeline_job_with_caching.json",
        pipeline_root=PIPELINE_ROOT,
        enable_caching=True,
    )
    job.run()


In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(pipeline_func=pipeline, package_path="intro_pipeline.json")

In [ ]:
DISPLAY_NAME = "intro_pipeline_job_unique"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="intro_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

In [ ]:
job.delete()

In [ ]:
job = aip.PipelineJob(
    display_name="intro_pipeline_job_cached_unique",
    template_path="intro_pipeline.json",
    enable_caching=True,
)

job.run()

In [ ]:
job.delete()